<a href="https://colab.research.google.com/github/Natkitta252603/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/MIDmit')

!pwd # เช็ค path ที่กำลังทำงานอยู่

/content/drive/MyDrive/MIDmit


In [ ]:
Data_House = pd.read_excel("/content/drive/MyDrive/MIDmit/Data House.xlsx")
Data_House

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Data_House.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
Data_House.shape

(1656, 47)

In [ ]:
Data_House.columns

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden'],
      dtype='object')

In [ ]:
# prompt: อ่านไฟล์json

import json

# Assuming '7-eleven.json' and 'school.json' are in the same directory
# Replace with the actual filepaths if needed
try:
    with open('/content/drive/MyDrive/MIDmit/7-eleven.json', 'r') as f:
        Data_7eleven = json.load(f)

    with open('/content/drive/MyDrive/MIDmit/school.json', 'r') as f:
        data_school = json.load(f)
except FileNotFoundError:
    print("Error: One or both JSON files not found. Please check the filepaths.")
    # Handle the error appropriately, e.g., exit the script or provide default values



In [ ]:
Data_7eleven

In [ ]:
data_school

#เตรียมข้อมูล

In [ ]:
Data_House.isnull().any() #เช็คค่าmissing

,0
id,False
name_th,True
name_en,True
project_name,True
company,True
province,False
district,False
sub_district,True
property_type,False
home improvement,True


In [ ]:
columns_to_check = ['renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va', 'number_of_story',
                   'number_of_bedroom', 'number_of_bathroom', 'direction', 'furniture', 'usable_area',
                   'number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']

# Check for missing values in each column and print the results
for col in columns_to_check:
    missing_count = Data_House[col].isnull().sum()
    print(f"Missing values in '{col}': {missing_count}")

Missing values in 'renovate': 211
Missing values in 'number_of_rai': 0
Missing values in 'number_of_ngan': 0
Missing values in 'number_of_va': 0
Missing values in 'number_of_story': 549
Missing values in 'number_of_bedroom': 27
Missing values in 'number_of_bathroom': 0
Missing values in 'direction': 0
Missing values in 'furniture': 0
Missing values in 'usable_area': 920
Missing values in 'number_of_parking': 215
Missing values in 'sell_price': 1
Missing values in 'district': 0
Missing values in 'sub_district': 140
Missing values in 'zone': 638


In [ ]:
# prompt: แทนค่าว่างดวยUnnkow เฉพาะ'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone' จากไฟล์ "Data House"

cols_to_fill = ['renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']
Data_House[cols_to_fill] = Data_House[cols_to_fill].fillna('Unknown')

In [ ]:
Data_House

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#เช็คค่าว่างอีกครั้งหลังแทนด้วยUnknow
columns_to_check = ['renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va', 'number_of_story',
                   'number_of_bedroom', 'number_of_bathroom', 'direction', 'furniture', 'usable_area',
                   'number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']

# Check for missing values in each column and print the results
for col in columns_to_check:
    missing_count = Data_House[col].isnull().sum()
    print(f"Missing values in '{col}': {missing_count}")

Missing values in 'renovate': 0
Missing values in 'number_of_rai': 0
Missing values in 'number_of_ngan': 0
Missing values in 'number_of_va': 0
Missing values in 'number_of_story': 0
Missing values in 'number_of_bedroom': 0
Missing values in 'number_of_bathroom': 0
Missing values in 'direction': 0
Missing values in 'furniture': 0
Missing values in 'usable_area': 0
Missing values in 'number_of_parking': 0
Missing values in 'sell_price': 0
Missing values in 'district': 0
Missing values in 'sub_district': 0
Missing values in 'zone': 0


In [ ]:
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic

# ... (Your existing code)

def count_nearby_places(house_lat, house_lon, places_data, radius_km):
    count = 0
    for place in places_data:
        place_lat = place['lat']
        place_lon = place['lon']
        distance = geodesic((house_lat, house_lon), (place_lat, place_lon)).km
        if distance <= radius_km:
            count += 1
    return count

# Create empty lists to store the counts
d1000_711_counts = []
d1000_school_counts = []

for index, row in Data_House.iterrows():
    try:
        house_id = row['id']  # Assuming 'id' column exists in Data_House
        house_lat = row['latitude']
        house_lon = row['longitude']


        # Find 7-11 count within 1 km
        d1000_711 = count_nearby_places(house_lat, house_lon, Data_7eleven, 1)
        d1000_711_counts.append(d1000_711)

        # Find school count within 1 km
        d1000_school = count_nearby_places(house_lat, house_lon, data_school, 1)
        d1000_school_counts.append(d1000_school)

    except (KeyError, TypeError) as e:
        print(f"Error processing row {index}: {e}")
        # Handle the error appropriately, e.g., append a default value or skip the row
        d1000_711_counts.append(0)  # or np.nan
        d1000_school_counts.append(0)  # or np.nan

# Add the new columns to the DataFrame
Data_House['d1000-7/11'] = d1000_711_counts
Data_House['d1000-school'] = d1000_school_counts

print(Data_House.head())

In [ ]:
# prompt: จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร

# ... (Your existing code)

def count_nearby_places(house_lat, house_lon, places_data, radius_km):
    count = 0
    for place in places_data:
        try:
            place_lat = float(place['lat'])
            place_lon = float(place['lon'])
            distance = geodesic((house_lat, house_lon), (place_lat, place_lon)).km
            if distance <= radius_km:
                count += 1
        except (KeyError, TypeError, ValueError):
            # Handle cases where 'lat' or 'lon' are missing or not numeric
            continue #skip if error
    return count

# Create empty lists to store the counts
d1000_711_counts = []
d1000_school_counts = []

for index, row in Data_House.iterrows():
    try:
        house_id = row['id']  # Assuming 'id' column exists in Data_House
        house_lat = float(row['latitude']) #convert latitude to float
        house_lon = float(row['longitude']) #convert longitude to float

        # Find 7-11 count within 1 km
        d1000_711 = count_nearby_places(house_lat, house_lon, Data_7eleven, 1)
        d1000_711_counts.append(d1000_711)

        # Find school count within 1 km
        d1000_school = count_nearby_places(house_lat, house_lon, data_school, 1)
        d1000_school_counts.append(d1000_school)

    except (KeyError, TypeError, ValueError) as e:
        print(f"Error processing row {index}: {e}")
        # Handle the error appropriately, e.g., append a default value or skip the row
        d1000_711_counts.append(0)  # or np.nan
        d1000_school_counts.append(0)  # or np.nan

# Add the new columns to the DataFrame
Data_House['d1000-7/11'] = d1000_711_counts
Data_House['d1000-school'] = d1000_school_counts

print(Data_House.head())

   id                                   name_th  \
0   1                   บ้านเดี่ยว ชนบท ขอนแก่น   
1   4                 บ้านเดี่ยว น้ำพอง ขอนแก่น   
2  16              บ้านเดี่ยว มัญจาคีรี ขอนแก่น   
3  18     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   
4  19  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม   

                                    name_en    project_name  \
0                   บ้านเดี่ยว ชนบท ขอนแก่น     บ้านวังเวิน   
1                 บ้านเดี่ยว น้ำพอง ขอนแก่น  บ้านหนองบัวบาน   
2              บ้านเดี่ยว มัญจาคีรี ขอนแก่น             NaN   
3     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   ชลลดา ขอนแก่น   
4  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม        เบสท์โฮม   

                                 company province      district sub_district  \
0                                    NaN  ขอนแก่น          ชนบท  ศรีบุญเรือง   
1                                    NaN  ขอนแก่น        น้ำพอง      ทรายมูล   
2                                    NaN  ขอนแก่น     มัญจาคีรี      กุดเค

##โจทย์ ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว

In [ ]:
Data_ID = pd.read_excel("/content/drive/MyDrive/MIDmit/คำตอบนักประเมิน(ID).xlsx")
Data_ID

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,965000.0,550000.0,"ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,1000000.0,1000000.0,"ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,10873000.0,10000000.0,"ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,1000000.0,1200000.0,ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [ ]:
# prompt: คอลัมม์ประเมินราคาซื้อ-ขายเปลี่ยนชื่อเป็นData_ID และ คอลัมม์ให้คะแนนทรัพย์เปลี่ยนชื่อเป็นscoreจากไฟล์ Data_ID

Data_ID = Data_ID.rename(columns={'ประเมินราคาซื้อ-ขาย': 'Data_ID', 'ให้คะแนนทรัพย์': 'score'})
Data_ID

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,Data_ID,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2700000.0,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2400000.0,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,965000.0,550000.0,"ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,1000000.0,1000000.0,"ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,10873000.0,10000000.0,"ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,1000000.0,1200000.0,ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


##ข้อ2จากตารางในข้อ 1 ตอบคำถามต่อไปนี้

#1.zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด

In [ ]:
# prompt: zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด

# Group by 'zone' and calculate the mean 'sell_price'
average_sell_price_by_zone = Data_House.groupby('zone')['sell_price'].mean()

# Find the zone with the highest average sell price
zone_with_highest_price = average_sell_price_by_zone.idxmax()

print(f"The zone with the highest average selling price is: {zone_with_highest_price}")

The zone with the highest average selling price is: ในเมือง-รื่นรมย์-เทพารักษ์


##2.บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่

##3.ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด

In [142]:
# prompt: sub_districtที่ ได้'score มากที่สุด

# Assuming merged_data DataFrame is already created as shown in the previous code

# Group by 'sub_district' and calculate the mean 'score'
average_score_by_subdistrict = merged_data.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest average score
subdistrict_with_highest_score = average_score_by_subdistrict.idxmax()

print(f"The sub_district with the highest average score is: {subdistrict_with_highest_score}")

NameError: name 'merged_data' is not defined